In [ ]:
import pandas as pd
import nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
ls = WordNetLemmatizer()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("Dataset.csv")              
print(df.shape)
tweets = df['text']
print(tweets)

(211746, 12)
0         How has #PTSD affected this writer's #quaranti...
1          #blackandyellow #Love #moments #september #Au...
2         Things I Do In My Spare Time Play Game 1- Eco ...
3         每天花5-30分钟进行正念冥想。使用此链接可帮助您开始冥想：https://www.apa....
4         Your #Quarantine #MovieList: THE HOUSE ON SORO...
                                ...                        
211741    A man in Gujranwala* got his ass kicked by his...
211742    @joobsie Astaga! Dua kali aku kena slap malam ...
211743    RT @sfrizwan: #ViolenceAgainstMen RT @AmazingS...
211744    #ViolenceAgainstMen RT @AmazingSusan Women rap...
211745    Spotlight On Male Victims Of Domestic Abuse ht...
Name: text, Length: 211746, dtype: object


In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
stop_words = set(stopwords.words('english'))
def cleaning(text):
    text = decontracted(text)
    text = text.lower()                              #lowering the text
    text = re.sub(r'@\S+','',text)                   #Removed @mentions
    text = re.sub(r'#\S+','',text)                   #Remove the hyper link
    text = re.sub(r'RT\S+','',text)                  #Removing ReTweets
    text = re.sub(r'https?\S+','',text)              #Remove the hyper link
    text = re.sub('[^a-z]',' ',text)              #Remove the character other than alphabet
    text = text.split()
    return text
tweets_cleaned =  tweets.apply(cleaning)
tweets_cleaned

0         [how, has, affected, this, writer, is, experie...
1                                                        []
2         [things, i, do, in, my, spare, time, play, gam...
3                                                        []
4         [your, the, house, on, sorority, row, my, review]
                                ...                        
211741    [a, man, in, gujranwala, got, his, ass, kicked...
211742    [astaga, dua, kali, aku, kena, slap, malam, ni...
211743                       [rt, rt, women, rape, men, in]
211744                           [rt, women, rape, men, in]
211745    [spotlight, on, male, victims, of, domestic, a...
Name: text, Length: 211746, dtype: object

In [ ]:
stop_words = set(stopwords.words('english'))
tweets_stemmed = tweets_cleaned.apply(lambda x: [ps.stem(word) for word in x if word not in stop_words])
tweets_lemmatized = tweets_cleaned.apply(lambda x: [ls.lemmatize(word) for word in x if word not in stop_words])

In [ ]:
tweets_stemmed = tweets_stemmed.apply(lambda x: ' '.join(x))
tweets_lemmatized = tweets_lemmatized.apply(lambda x: ' '.join(x))

In [ ]:
df['scrapped_text'] = df['text']
df['Lemmatized_text'] = tweets_lemmatized.to_frame() 
df['Stemmed_text'] = tweets_lemmatized.to_frame()
new_df=df.drop(['Unnamed: 0', 'id', 'permalink', 'username', 'to', 'text', 'date','retweets', 'favorites', 'mentions', 'hashtags', 'geo',],axis=1)

In [ ]:
new_df.head()

,scrapped_text,Lemmatized_text,Stemmed_text
0,How has #PTSD affected this writer's #quaranti...,affected writer experience click link find,affected writer experience click link find
1,#blackandyellow #Love #moments #september #Au...,,
2,Things I Do In My Spare Time Play Game 1- Eco ...,thing spare time play game eco unisex tee usd ...,thing spare time play game eco unisex tee usd ...
3,每天花5-30分钟进行正念冥想。使用此链接可帮助您开始冥想：https://www.apa....,,
4,Your #Quarantine #MovieList: THE HOUSE ON SORO...,house sorority row review,house sorority row review


In [ ]:
new_df.to_csv('New_Dataset_3.csv')

## Model Building Starts From here

In [ ]:
import pandas as pd
import nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
ls = WordNetLemmatizer()

In [ ]:
df=pd.read_csv('New_Dataset_3.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna(axis=0)

In [ ]:
df.shape

## Creating Stemmed AND Lemmatized text Dataset

In [ ]:
stem_df=df[['Stemmed_text']]
lemm_df=df[['Lemmatized_text']]

In [ ]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid=SentimentIntensityAnalyzer()

In [ ]:
stem_df['scores']=stem_df['Stemmed_text'].apply(lambda Stemmed_text :sid.polarity_scores(Stemmed_text))

In [ ]:
stem_df['compound']=stem_df['scores'].apply(lambda score_dict:score_dict['compound'])

In [ ]:
stem_df['comp_score']=stem_df['compound'].apply(lambda c:'pos' if c>=0 else "neg")

In [ ]:
stem_df.head()

In [ ]:
stem_df['comp_score'].value_counts()

In [ ]:
## Try to play with this number and check how much it work in bag of words 
stem_df=stem_df.iloc[:20000]

In [ ]:
stem_df.shape

## Implementing Bag of Words.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

X=cv.fit_transform(stem_df['Stemmed_text']).toarray()
Y=pd.get_dummies(stem_df['comp_score'])
Y=Y.iloc[:,1].values

In [ ]:
Y

In [ ]:
X.shape,Y.shape

## Train Test Split

In [ ]:
from sklearn.model_selection  import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

## Naive Bayes 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB().fit(x_train,y_train)

In [ ]:
y_pred=model.predict(x_test)

## Finding accuracy_score and printing confusion_matrix and classification_report

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
accuracy_score(y_test,y_pred)  #Testing Score

In [ ]:
accuracy_score(y_train, model.predict(x_train))  #Training Score

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

## Predicting a new tweet sentiments.

In [ ]:
df=pd.DataFrame(["i am tweeting because there is domestic violence spreading everywhere"],columns=['new_tweet'])

In [ ]:
df   

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

nltk.download('stopwords')
def cleaning(text):
    corpus = []
    text = decontracted(text)
    text = text.lower()                              #lowering the text
    text = re.sub(r'@\S+','',text)                   #Removed @mentions
    text = re.sub(r'#\S+','',text)                   #Remove the hyper link
    text = re.sub(r'RT\S+','',text)                  #Removing ReTweets
    text = re.sub(r'https?\S+','',text)              #Remove the hyper link
    text = re.sub('[^a-z]',' ',text)              #Remove the character other than alphabet
    text = text.split()
    text=[ps.stem(word) for word in text if word not in stopwords.words('english')]
    text=' '.join(text)
    corpus.append(text)
    return corpus

In [ ]:
tweet=df['new_tweet']
tweet_cleaned =  cleaning(tweet[0])
type(tweet_cleaned)

In [ ]:
df=cv.transform(tweet_cleaned).toarray()

pred=model.predict(df)

label=pred[0]

if label==1:
  print('Positive')
else:
  print('Negative')

## Saving the model

In [ ]:
from pickle import dump

dump(cv,open('pickle/countvectorizer.pkl','wb'))

dump(model,open('pickle/model.pkl','wb'))

In [ ]:
## Predicting new tweet

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

nltk.download('stopwords')
def cleaning(text):
    corpus = []
    text = decontracted(text)
    text = text.lower()                              #lowering the text
    text = re.sub(r'@\S+','',text)                   #Removed @mentions
    text = re.sub(r'#\S+','',text)                   #Remove the hyper link
    text = re.sub(r'RT\S+','',text)                  #Removing ReTweets
    text = re.sub(r'https?\S+','',text)              #Remove the hyper link
    text = re.sub('[^a-z]',' ',text)              #Remove the character other than alphabet
    text = text.split()
    text=[ps.stem(word) for word in text if word not in stopwords.words('english')]
    text=' '.join(text)
    corpus.append(text)
    return corpus

In [ ]:
from pickle import load

def predict(input_msg):
    vectorizer = load(open('pickle/countvectorizer.pkl','rb'))
    
    classifier = load(open('pickle/model.pkl','rb'))
    
    clean_text = cleaning(input_msg)
    
    clean_text_encoded = vectorizer.transform(clean_text)
    
    future_text = clean_text_encoded.toarray()
    
    prediction = classifier.predict(future_text)
    
    return prediction

In [ ]:
input_tweet = input()

In [ ]:
prediction = predict(input_tweet)

In [ ]:
if prediction == 1:
  print('Positive')
else:
  print('Negative')

Positive
